# **WBS Template Duration Analysis**

#### *If you need to install packages below, remove '#' and run the code below*

In [ ]:
#install.packages("ggplot2")
#install.packages("moments")
#install.packages("mltools")
#install.packages("caret")
#install.packages("writexl")
#install.packages("reshape2")
#install.packages("dplyr")

#### *Load required libraries*

In [ ]:
library(readxl)
library(dplyr)
library(mltools)
library(caret)
library(data.table)
library(reshape2)
library(writexl)
library(ggplot2)
library(moments)
library(MASS)
library(reshape2)

#### *Enter your file path and run the code below*

In [ ]:
# Read Excel
MITB_CELL_S <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\소형_S_ver3.xlsx")
MITB_CELL_A <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\소형_A_ver3.xlsx")
MITB_CELL_B <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\소형_B_ver3.xlsx")

EV_CELL_S <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\자동차_S_ver3.xlsx")
EV_CELL_A <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\자동차_A_ver3.xlsx")
EV_CELL_B <- read_excel("C:\\Users\\LG\\Desktop\\IBT\\배치부서특화\\과제_8월\\WBS일정계획\\raw data\\2차추출_all\\final_dataset\\자동차_B_ver3.xlsx")

# View(MITB_CELL_S) # not supported in Jupyter R Kernel
table(MITB_CELL_S$PERIOD_DAYS)

In [ ]:
length(EV_CELL_S) #63
#length(EV_CELL_A) #63
#length(EV_CELL_B) #63

#length(MITB_CELL_S) #63
#length(MITB_CELL_A) #63
#length(MITB_CELL_B) #63

nrow(EV_CELL_S) #54
nrow(EV_CELL_A) #41
nrow(EV_CELL_B) #84
# total: 179
54+41+84

nrow(MITB_CELL_S) #170
nrow(MITB_CELL_A) #189
nrow(MITB_CELL_B) #500
# total: 859
170+189+500

# View(MITB_CELL_S) # 671 -> 602 # not supported in Jupyter R Kernel

## **Data Preprocessing**

#### *Assign the Model Grade & Business Unit Code*

In [ ]:
EV_CELL_S[ , "MODEL_GRADE"] <- "S"
EV_CELL_A[ , "MODEL_GRADE"] <- "A"
EV_CELL_B[ , "MODEL_GRADE"] <- "B"

MITB_CELL_S[ , "MODEL_GRADE"] <- "S"
MITB_CELL_A[ , "MODEL_GRADE"] <- "A"
MITB_CELL_B[ , "MODEL_GRADE"] <- "B"

EV_CELL_S[ , "BUSINESS_UNIT_CODE"] <- "EV"
EV_CELL_A[ , "BUSINESS_UNIT_CODE"] <- "EV"
EV_CELL_B[ , "BUSINESS_UNIT_CODE"] <- "EV"

MITB_CELL_S[ , "BUSINESS_UNIT_CODE"] <- "MITB"
MITB_CELL_A[ , "BUSINESS_UNIT_CODE"] <- "MITB"
MITB_CELL_B[ , "BUSINESS_UNIT_CODE"] <- "MITB"

length(MITB_CELL_S) #63 -> 65
nrow(MITB_CELL_S) #170

#### *Check the Type of PERIOD_DAYS and PREIOD_ACTUAL*

In [ ]:
# EV, MITB 모두 PERIOD_ACTUAL 만 Character type 임
head(EV_CELL_S$PERIOD_DAYS)
#EV_CELL_A$PERIOD_DAYS
#EV_CELL_B$PERIOD_DAYS

#MITB_CELL_S$PERIOD_DAYS
#MITB_CELL_A$PERIOD_DAYS
#MITB_CELL_B$PERIOD_DAYS

#EV_CELL_S$PERIOD_ACTUAL
#EV_CELL_A$PERIOD_ACTUAL
#EV_CELL_B$PERIOD_ACTUAL

#MITB_CELL_S$PERIOD_ACTUAL
#MITB_CELL_A$PERIOD_ACTUAL
#MITB_CELL_B$PERIOD_ACTUAL # Advanced Copy

#### *Delete "[NULL]" (character type)*

In [ ]:
EV_CELL_S <- EV_CELL_S[!(EV_CELL_S$PERIOD_ACTUAL == "[NULL]" ), ]
EV_CELL_A <- EV_CELL_A[!(EV_CELL_A$PERIOD_ACTUAL == "[NULL]" ), ]
EV_CELL_B <- EV_CELL_B[!(EV_CELL_B$PERIOD_ACTUAL == "[NULL]" ), ]

MITB_CELL_S <- MITB_CELL_S[!(MITB_CELL_S$PERIOD_ACTUAL == "[NULL]" ), ]
MITB_CELL_A <- MITB_CELL_A[!(MITB_CELL_A$PERIOD_ACTUAL == "[NULL]" ), ]
MITB_CELL_B <- MITB_CELL_B[!(MITB_CELL_B$PERIOD_ACTUAL == "[NULL]" ), ]

#### *Change Data Type of Period*

In [ ]:
EV_CELL_S$PERIOD_ACTUAL <- as.numeric(EV_CELL_S$PERIOD_ACTUAL)
EV_CELL_A$PERIOD_ACTUAL <- as.numeric(EV_CELL_A$PERIOD_ACTUAL)
EV_CELL_B$PERIOD_ACTUAL <- as.numeric(EV_CELL_B$PERIOD_ACTUAL)

MITB_CELL_S$PERIOD_ACTUAL <- as.numeric(MITB_CELL_S$PERIOD_ACTUAL)
MITB_CELL_A$PERIOD_ACTUAL <- as.numeric(MITB_CELL_A$PERIOD_ACTUAL)
MITB_CELL_B$PERIOD_ACTUAL <- as.numeric(MITB_CELL_B$PERIOD_ACTUAL)

#### *Check the type of PERIOD_ACTUAL and PERIOD_DAYS again*

In [ ]:
# all double
typeof(EV_CELL_S$PERIOD_DAYS) # character -> double
typeof(EV_CELL_A$PERIOD_ACTUAL) # character -> double
typeof(EV_CELL_B$PERIOD_DAYS) # character -> double

typeof(MITB_CELL_S$PERIOD_ACTUAL) # character -> double
typeof(MITB_CELL_A$PERIOD_DAYS) # character -> double
typeof(MITB_CELL_B$PERIOD_ACTUAL) # character -> double

#### *Run the code below when you need to delete NA Values*

In [ ]:
EV_CELL_S <- EV_CELL_S %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))
EV_CELL_A <- EV_CELL_A %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))
EV_CELL_B <- EV_CELL_B %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))

MITB_CELL_S <- MITB_CELL_S %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))
MITB_CELL_A <- MITB_CELL_A %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))
MITB_CELL_B <- MITB_CELL_B %>% filter(!is.na(PERIOD_ACTUAL) & !is.na(PERIOD_DAYS))

#### *Check the presence of NA for the PERIOD_ACTUAL and PERIOD_DAYS*

In [ ]:
# all zero
sum(is.na(EV_CELL_S$PERIOD_ACTUAL)) #0
sum(is.na(EV_CELL_A$PERIOD_ACTUAL)) #0
sum(is.na(EV_CELL_B$PERIOD_ACTUAL)) #0

sum(is.na(MITB_CELL_S$PERIOD_ACTUAL)) #0
sum(is.na(MITB_CELL_A$PERIOD_ACTUAL)) #0
sum(is.na(MITB_CELL_B$PERIOD_ACTUAL)) #0

sum(is.na(EV_CELL_S$PERIOD_DAYS)) #0
sum(is.na(EV_CELL_A$PERIOD_DAYS)) #0
sum(is.na(EV_CELL_B$PERIOD_DAYS)) #0

sum(is.na(MITB_CELL_S$PERIOD_DAYS)) #0 
sum(is.na(MITB_CELL_A$PERIOD_DAYS)) #0
sum(is.na(MITB_CELL_B$PERIOD_DAYS)) #0

#### *Calculate the differences between Period_Plan and Period_Actual*

In [ ]:
EV_CELL_S[, "Diff_Period"] <- EV_CELL_S$PERIOD_ACTUAL - EV_CELL_S$PERIOD_DAYS
EV_CELL_A[, "Diff_Period"] <- EV_CELL_A$PERIOD_ACTUAL - EV_CELL_A$PERIOD_DAYS
EV_CELL_B[, "Diff_Period"] <- EV_CELL_B$PERIOD_ACTUAL - EV_CELL_B$PERIOD_DAYS

MITB_CELL_S[, "Diff_Period"] <- MITB_CELL_S$PERIOD_ACTUAL - MITB_CELL_S$PERIOD_DAYS
MITB_CELL_A[, "Diff_Period"] <- MITB_CELL_A$PERIOD_ACTUAL - MITB_CELL_A$PERIOD_DAYS
MITB_CELL_B[, "Diff_Period"] <- MITB_CELL_B$PERIOD_ACTUAL - MITB_CELL_B$PERIOD_DAYS

#### *Check the presence of NA for the Diff_Period*

In [ ]:
# all zero
sum(is.na(EV_CELL_S$Diff_Period)) #0 
sum(is.na(EV_CELL_A$Diff_Period)) #0
sum(is.na(EV_CELL_B$Diff_Period)) #0

sum(is.na(MITB_CELL_S$Diff_Period)) #0 
sum(is.na(MITB_CELL_A$Diff_Period)) #0
sum(is.na(MITB_CELL_B$Diff_Period)) #0

In [ ]:
# 66 is normal
length(EV_CELL_S) #63 -> 66
length(EV_CELL_A) #63 -> 66
length(EV_CELL_B) #63 -> 66

length(MITB_CELL_S) #63 -> 66
length(MITB_CELL_A) #63 -> 66
length(MITB_CELL_B) #63 -> 66

#### *Count the number of the rows*

In [ ]:
nrow(EV_CELL_S) #49
nrow(EV_CELL_A) #37
nrow(EV_CELL_B) #82

nrow(MITB_CELL_S) #154
nrow(MITB_CELL_A) #174
nrow(MITB_CELL_B) #490

#### *Select the required columns*

In [ ]:
names(EV_CELL_S)

EV_CELL_S <- EV_CELL_S[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                           "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                           , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]

EV_CELL_A <- EV_CELL_A[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                           "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                           , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]

EV_CELL_B <- EV_CELL_B[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                           "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                           , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]

MITB_CELL_S <- MITB_CELL_S[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                               "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                               , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]


MITB_CELL_A <- MITB_CELL_A[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                               "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                               , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]

MITB_CELL_B <- MITB_CELL_B[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                               "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "RESULT_START_DATE"
                               , "PERIOD_ACTUAL", "MILESTONE", "ACTIVITY_TYPE_NAME", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "Diff_Period")]
# "View" is not supported in Jupyter R Kernel
# View(EV_CELL_S)
# View(EV_CELL_A)
# View(EV_CELL_B)

# View(MITB_CELL_S)
# View(MITB_CELL_A)
# View(MITB_CELL_B)

#### *Check the column count for the new data*

In [ ]:
# 18 is normal
length(EV_CELL_S) #65 -> 18
length(EV_CELL_A) #65 -> 18
length(EV_CELL_B) #65 -> 18

length(MITB_CELL_S) #65 -> 18
length(MITB_CELL_A) #65 -> 18
length(MITB_CELL_B) #65 -> 18

#### *Date Formatting form POSIXIct to UTC Form*

In [ ]:
head(EV_CELL_S$RESULT_START_DATE)
#EV_CELL_A$RESULT_START_DATE
#EV_CELL_B$RESULT_START_DATE

#MITB_CELL_S$RESULT_START_DATE
#MITB_CELL_A$RESULT_START_DATE
#MITB_CELL_B$RESULT_START_DATE

In [ ]:
# all double
typeof(EV_CELL_S$RESULT_START_DATE)
typeof(EV_CELL_A$RESULT_START_DATE)
typeof(EV_CELL_B$RESULT_START_DATE)

typeof(MITB_CELL_S$RESULT_START_DATE)
typeof(MITB_CELL_A$RESULT_START_DATE)
typeof(MITB_CELL_B$RESULT_START_DATE)

In [ ]:
# dttm -> date
EV_CELL_S[, "RESULT_START_DATE"] <- as.Date(EV_CELL_S$RESULT_START_DATE)
EV_CELL_A[, "RESULT_START_DATE"] <- as.Date(EV_CELL_A$RESULT_START_DATE)
EV_CELL_B[, "RESULT_START_DATE"] <- as.Date(EV_CELL_B$RESULT_START_DATE)

MITB_CELL_S[, "RESULT_START_DATE"] <- as.Date(MITB_CELL_S$RESULT_START_DATE)
MITB_CELL_A[, "RESULT_START_DATE"] <- as.Date(MITB_CELL_A$RESULT_START_DATE)
MITB_CELL_B[, "RESULT_START_DATE"] <- as.Date(MITB_CELL_B$RESULT_START_DATE)

#### *rbind to make our final_data_EV and final_data_MITB*

In [ ]:
final_data_EV <- rbind(EV_CELL_S, EV_CELL_A)
final_data_EV <- rbind(final_data_EV, EV_CELL_B)

final_data_MITB <- rbind(MITB_CELL_S, MITB_CELL_A)
final_data_MITB <- rbind(final_data_MITB, MITB_CELL_B)

#### *info of the final_data*

In [ ]:
# View(final_data_EV) # not supported in Jupyter R Kernel
length(final_data_EV) #18
nrow(final_data_EV) #168

# View(final_data_MITB) # not supported in Jupyter R Kernel
length(final_data_MITB) #18
nrow(final_data_MITB) #818
colnames(final_data_MITB)

#### *delete the '[NULL]' (character) in the final_data*

In [ ]:
final_data_EV <- final_data_EV[!(final_data_EV$ACTIVITY_TYPE_NAME == "[NULL]" ), ]
final_data_MITB <- final_data_MITB[!(final_data_MITB$ACTIVITY_TYPE_NAME == "[NULL]" ), ]

#### *type of the ACTIVITY TYPE NAME*

In [ ]:
typeof(final_data_EV$ACTIVITY_TYPE_NAME) # character
typeof(final_data_MITB$ACTIVITY_TYPE_NAME) # character

#### *Only Keep CV, DV, PD, PV in the Activity Type Name*

In [ ]:
final_data_EV <- final_data_EV[(final_data_EV$ACTIVITY_TYPE_NAME == 'CV' | final_data_EV$ACTIVITY_TYPE_NAME == 'DV' 
                                | final_data_EV$ACTIVITY_TYPE_NAME == 'PD' | final_data_EV$ACTIVITY_TYPE_NAME == 'PV'), ]

final_data_MITB <- final_data_MITB[(final_data_MITB$ACTIVITY_TYPE_NAME == 'CV' | final_data_MITB$ACTIVITY_TYPE_NAME == 'DV' 
                                    | final_data_MITB$ACTIVITY_TYPE_NAME == 'PD' | final_data_MITB$ACTIVITY_TYPE_NAME == 'PV'), ]

#### *Delete NA values in the Activity Type Name*

In [ ]:
final_data_EV <- final_data_EV %>% filter(!is.na(ACTIVITY_TYPE_NAME))
final_data_MITB <- final_data_MITB %>% filter(!is.na(ACTIVITY_TYPE_NAME))

head(final_data_EV$ACTIVITY_TYPE_NAME, 5)
head(final_data_MITB$ACTIVITY_TYPE_NAME, 5)

In [ ]:
length(final_data_EV) #18
nrow(final_data_EV) #127
# View(final_data_EV) # not supported in Jupyter R Kernel

length(final_data_MITB) #18
nrow(final_data_MITB) #614
# View(final_data_MITB) # not supported in Jupyter R Kernel

#### *encoding the column of ACTIVITY TYPE, MODEL_GRADE*

In [ ]:
final_data_encoding_EV <- dcast(data=final_data_EV, PJT_OBID+ACTIVITY_ID ~ MODEL_GRADE+ACTIVITY_TYPE_NAME, length)
final_data_encoding_MITB <- dcast(data=final_data_MITB, PJT_OBID+ACTIVITY_ID ~ MODEL_GRADE+ACTIVITY_TYPE_NAME, length)

#### *join the final_data_encoding (EV & MITB) and the origin data*

In [ ]:
final_join_data_EV <- inner_join(final_data_EV, final_data_encoding_EV, by=c('PJT_OBID', 'ACTIVITY_ID'))
final_join_data_MITB <- inner_join(final_data_MITB, final_data_encoding_MITB, by=c('PJT_OBID', 'ACTIVITY_ID'))

length(final_join_data_EV) #30
nrow(final_join_data_EV) #127
View(final_join_data_EV) # not supported in Jupyter R Kernel

length(final_join_data_MITB) #30
nrow(final_join_data_MITB) #614
View(final_join_data_MITB) # not supported in Jupyter R Kernel

#### *Select only the required column*

In [ ]:
names(final_join_data_EV)
names(final_join_data_MITB)

final_join_data_EV <- final_join_data_EV[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                                             "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "ACTIVITY_TYPE_NAME"
                                             , "PERIOD_ACTUAL", "MILESTONE", "RESULT_START_DATE", "Diff_Period", "S_CV", "S_DV", "S_PD", "S_PV", "A_CV", "A_DV", "A_PD", "A_PV", "B_CV", "B_DV", "B_PD", "B_PV")]

final_join_data_MITB <- final_join_data_MITB[, c("PJT_OBID", "ACTIVITY_ID", "MODEL_NAME", "ACTIVITY_TYPE_CODE", "SORT_ORDER_NO", "LEVEL", "LV_DEPTH", 
                                                 "IS_CHK_EXIST_CHILD", "ACTIVITY_TITLE", "PERIOD_DAYS", "EXE_RATE", "MODEL_GRADE", "BUSINESS_UNIT_CODE", "ACTIVITY_TYPE_NAME"
                                                 , "PERIOD_ACTUAL", "MILESTONE", "RESULT_START_DATE", "Diff_Period", "S_CV", "S_DV", "S_PD", "S_PV", "A_CV", "A_DV", "A_PD", "A_PV", "B_CV", "B_DV", "B_PD", "B_PV")]

## **Exploratory Data Analysis**

### **1. str**

In [ ]:
str(final_join_data_EV)
str(final_join_data_MITB)

names(final_join_data_EV)
names(final_join_data_MITB)

#### *Calculate the Skewness*

In [ ]:
skewness(final_join_data_EV$PERIOD_DAYS)
skewness(final_join_data_EV$PERIOD_ACTUAL)

skewness(final_join_data_MITB$PERIOD_DAYS)
skewness(final_join_data_MITB$PERIOD_ACTUAL)

#### *Histogram to check the Probability Density*

In [ ]:
# Auto-mobile
hist(final_join_data_EV$PERIOD_DAYS, breaks=25, probability=TRUE, main='PDF of PLANNING_PERIOD \n (Auto Mobile)', 
     col='Navy', border='white', xlab='Right Skewed')
lines(density(final_join_data_EV$PERIOD_DAYS), col='red') 

hist(final_join_data_EV$PERIOD_ACTUAL, breaks=25, probability=TRUE, main='PDF of ACTUAL_PERIOD \n (Auto Mobile)', 
     col='Navy', border='white', xlab='Right Skewed')
lines(density(final_join_data_EV$PERIOD_ACTUAL), col='red')

# Mobile & IT
hist(final_join_data_MITB$PERIOD_DAYS, breaks=25, probability=TRUE, main='PDF of PLANNING_PERIOD \n (Mobile & IT)', 
     col='Navy', border='white', xlab='Right Skewed')
lines(density(final_join_data_MITB$PERIOD_DAYS), col='red')

hist(final_join_data_MITB$PERIOD_ACTUAL, breaks=25, probability=TRUE, main='PDF of ACTUAL_PERIOD \n (Mobile & IT)', 
     col='Navy', border='white', xlab='Right Skewed')
lines(density(final_join_data_MITB$PERIOD_ACTUAL), col='red')

#### *Compare with the Normal Distribution*

In [ ]:
normx <- rnorm(n=10000, mean=0, sd=1)
qqnorm(normx)
qqline(normx, col='red')

# Automobile
qqnorm(final_join_data_EV$PERIOD_DAYS, main='QQPlot of PLANNING_PERIOD \n (Auto Mobile)', col='Navy', 
       xlab='Right Skewed')
qqline(final_join_data_EV$PERIOD_DAYS, col='red')

qqnorm(final_join_data_EV$PERIOD_ACTUAL, main='QQPlot of ACTUAL_PERIOD \n (Auto Mobile)', col='Navy', 
       xlab='Right Skewed')
qqline(final_join_data_EV$PERIOD_ACTUAL, col='red')

# Mobile & IT
qqnorm(final_join_data_MITB$PERIOD_DAYS, main='QQPlot of PLANNING_PERIOD \n (Mobile & IT)', col='Navy', 
       xlab='Right Skewed')
qqline(final_join_data_MITB$PERIOD_DAYS, col='red')

qqnorm(final_join_data_MITB$PERIOD_ACTUAL, main='QQPlot of ACTUAL_PERIOD \n (Mobile & IT)', col='Navy', 
       xlab='Right Skewed')
qqline(final_join_data_MITB$PERIOD_ACTUAL, col='red')

#### *cannot reject H_0*

In [ ]:
shapiro.test(final_join_data_EV$PERIOD_ACTUAL) # p-value = 1.021e-05
shapiro.test(final_join_data_EV$PERIOD_DAYS) # p-value = 1.585e-06

shapiro.test(final_join_data_MITB$PERIOD_ACTUAL)# p-value < 2.2e-16
shapiro.test(final_join_data_MITB$PERIOD_DAYS) # p-value < 2.2e-16

### **2. scatter plot**

In [ ]:
plot(final_join_data_EV[,c(10, 11, 15, 16)]) # 뚜렷한 상관관계 or insight 없음

###  **3. box plot of PERIOD_ACTUAL & PERIOD_DAYS for extract abnormal data**

In [ ]:
# Auto Mobile
Boxplot_EV_PLAN <- boxplot(final_join_data_EV$PERIOD_DAYS, col='Navy', main='BOX PLOT of PLANNING PERIOD \n (Auto Mobile)', 
                           xlab="7 Outliers Found \n (more than 1088 days)")
Boxplot_EV_ACTUAL <- boxplot(final_join_data_EV$PERIOD_ACTUAL, col='Navy', main='BOX PLOT of ACTUAL PERIOD \n (Auto Mobile)', 
                             xlab="4 Outliers Found \n (more than 830 days)")

# Mobile & IT
Boxplot_MITB_PLAN <- boxplot(final_join_data_MITB$PERIOD_DAYS, col='Navy', main='BOX PLOT of PLANNING PERIOD \n (Mobile & IT)', 
                             xlab="111 Outliers Found \n (more than 194 days)")
Boxplot_MITB_ACTUAL <- boxplot(final_join_data_MITB$PERIOD_ACTUAL, col='Navy', main='BOX PLOT of ACTUAL PERIOD \n (Mobile & IT)', 
                               xlab="42 Outliers Found \n (more than 196 days)")

#### *Check the outlier & normal range*

In [ ]:
# Auto mobile
boxplot(final_join_data_EV$PERIOD_DAYS)$stats # 1 ~ 1088
boxplot(final_join_data_EV$PERIOD_ACTUAL)$stats # 0 ~ 830

# Mobile & IT
boxplot(final_join_data_MITB$PERIOD_DAYS)$stats # 1 ~ 334 -> 247 -> 222 -> 210 -> 207 -> 200 -> 194 # 111개
boxplot(final_join_data_MITB$PERIOD_ACTUAL)$stats # 0 ~ 239 -> 208 -> 196 # 42개

#### *Delete the outliers*

In [ ]:
# Auto Mobile
final_join_data_EV <- final_join_data_EV[!(final_join_data_EV$PERIOD_DAYS >= 1088), ]
final_join_data_EV <- final_join_data_EV[!(final_join_data_EV$PERIOD_ACTUAL >= 830), ]

# Mobile & IT
final_join_data_MITB <- final_join_data_MITB[!(final_join_data_MITB$PERIOD_DAYS >= 194), ]
final_join_data_MITB <- final_join_data_MITB[!(final_join_data_MITB$PERIOD_ACTUAL >= 196), ]

nrow(final_join_data_EV) # 127 -> 117 -> 116 # 11건 감소
nrow(final_join_data_MITB) # 614 -> 464 -> 416 # 198건 감소

#### *re-plotting*

In [ ]:
# Auto Mobile
Boxplot_EV_PLAN <- boxplot(final_join_data_EV$PERIOD_DAYS, col='Navy', main='BOX PLOT of PLANNING PERIOD \n (Auto Mobile)', 
                           xlab="0 Outliers Found \n (Trim once)")
Boxplot_EV_ACTUAL <- boxplot(final_join_data_EV$PERIOD_ACTUAL, col='Navy', main='BOX PLOT of ACTUAL PERIOD \n (Auto Mobile)', 
                             xlab="0 Outliers Found \n (Trim once)")

# Mobile & IT
Boxplot_MITB_PLAN <- boxplot(final_join_data_MITB$PERIOD_DAYS, col='Navy', main='BOX PLOT of PLANNING PERIOD \n (Mobile & IT)', 
                             xlab="0 Outliers Found \n (Trim 7 times)")
Boxplot_MITB_ACTUAL <- boxplot(final_join_data_MITB$PERIOD_ACTUAL, col='Navy', main='BOX PLOT of ACTUAL PERIOD \n (Mobile & IT)', 
                               xlab="0 Outliers Found \n (Trim 3 times)")

150/614
10/127

### **4. Check the NA**

#### *in the MODEL GRADE & ACTIVITY TYPE NAME*

In [ ]:
sum(is.na(final_data_EV$MODEL_GRADE)) #0
sum(is.na(final_data_EV$ACTIVITY_TYPE_NAME)) #0

sum(is.na(final_join_data_MITB$MODEL_GRADE)) #0
sum(is.na(final_join_data_MITB$ACTIVITY_TYPE_NAME)) #0

#### *in the dummy var*

In [ ]:
# Automobile
sum(is.na(final_join_data_EV$S_CV)) #0
sum(is.na(final_join_data_EV$S_DV)) #0
sum(is.na(final_join_data_EV$S_PD)) #0
sum(is.na(final_join_data_EV$S_PV)) #0
sum(is.na(final_join_data_EV$A_CV)) #0
sum(is.na(final_join_data_EV$A_DV)) #0
sum(is.na(final_join_data_EV$A_PD)) #0
sum(is.na(final_join_data_EV$A_PV)) #0
sum(is.na(final_join_data_EV$B_CV)) #0
sum(is.na(final_join_data_EV$B_DV)) #0
sum(is.na(final_join_data_EV$B_PD)) #0
sum(is.na(final_join_data_EV$B_PV)) #0

# Mobile & IT
sum(is.na(final_join_data_MITB$S_CV)) #0
sum(is.na(final_join_data_MITB$S_DV)) #0
sum(is.na(final_join_data_MITB$S_PD)) #0
sum(is.na(final_join_data_MITB$S_PV)) #0
sum(is.na(final_join_data_MITB$A_CV)) #0
sum(is.na(final_join_data_MITB$A_DV)) #0
sum(is.na(final_join_data_MITB$A_PD)) #0
sum(is.na(final_join_data_MITB$A_PV)) #0
sum(is.na(final_join_data_MITB$B_CV)) #0
sum(is.na(final_join_data_MITB$B_DV)) #0
sum(is.na(final_join_data_MITB$B_PD)) #0
sum(is.na(final_join_data_MITB$B_PV)) #0

### **5. histogram of PERIOD_ACTUAL & PERIOD_DAYS**

#### *re-plotting: Hypothesis Test for Normal Distribution*

In [ ]:
# Auto Mobile
Hist_EV_PLAN <- hist(final_join_data_EV$PERIOD_ACTUAL, breaks=25, main='Histogram of PLANNING_PERIOD \n (Auto Mobile)', col='Navy', 
                     border='Dark Slate Blue', xlab='Right Skewed')
Hist_EV_ACTUAL <- hist(final_join_data_EV$PERIOD_DAYS, breaks=25, main='Histogram of ACTUAL_PERIOD \n (Auto Mobile)', col='Navy', 
                       border='Dark Slate Blue', xlab='Right Skewed')

# Mobile & IT
Hist_MITB_PLAN <- hist(final_join_data_MITB$PERIOD_ACTUAL, breaks=25, main='Histogram of PLANNING_PERIOD \n (Mobile & IT)', 
                       col='Navy', border='Dark Slate Blue', xlab='Right Skewed')
Hist_MITB_ACTUAL <- hist(final_join_data_MITB$PERIOD_DAYS, breaks=25, main='Histogram of ACTUAL_PERIOD \n (Mobile & IT)', 
                         col='Navy', border='Dark Slate Blue', xlab='Right Skewed')

#### *re-calculate the Skewness*

In [ ]:
skewness(final_join_data_EV$PERIOD_DAYS)
skewness(final_join_data_EV$PERIOD_ACTUAL)

skewness(final_join_data_MITB$PERIOD_DAYS)
skewness(final_join_data_MITB$PERIOD_ACTUAL)

#### *re-plot: Histogram to check the Probability Density*

In [ ]:
# Add the line graph on the plot --> next trial!
hist(final_join_data_EV$PERIOD_DAYS, breaks=25, probability=TRUE, main='PDF of PLANNING_PERIOD \n (Auto Mobile)', 
     col='Navy', border='white', xlab='Light Tailed')
lines(density(final_join_data_EV$PERIOD_DAYS), col='red') 

hist(final_join_data_EV$PERIOD_ACTUAL, breaks=25, probability=TRUE, main='PDF of ACTUAL_PERIOD \n (Auto Mobile)', 
     col='Navy', border='white', xlab='Light Tailed')
lines(density(final_join_data_EV$PERIOD_ACTUAL), col='red')

hist(final_join_data_MITB$PERIOD_DAYS, breaks=25, probability=TRUE, main='PDF of PLANNING_PERIOD \n (Mobile & IT)', 
     col='Navy', border='white', xlab='Light Tailed')
lines(density(final_join_data_MITB$PERIOD_DAYS), col='red')

hist(final_join_data_MITB$PERIOD_ACTUAL, breaks=25, probability=TRUE, main='PDF of ACTUAL_PERIOD \n (Mobile & IT)', 
     col='Navy', border='white', xlab='Light Tailed')
lines(density(final_join_data_MITB$PERIOD_ACTUAL), col='red')

#### *re-compare with the Normal Distribution*

In [ ]:
normx <- rnorm(n=10000, mean=0, sd=1)
hist(normx, breaks=25, probability=TRUE, main='PDF of Normal Distribution',
     col='Navy', border='white', xlab='Normal')
lines(density(normx), col='red')

qqnorm(normx, col='Navy')
qqline(normx, col='red')

In [ ]:
# Automobile
qqnorm(final_join_data_EV$PERIOD_DAYS, main='QQPlot of PLANNING_PERIOD \n (Auto Mobile)', col='Navy', 
       xlab='Light Tailed')
qqline(final_join_data_EV$PERIOD_DAYS, col='red')

qqnorm(final_join_data_EV$PERIOD_ACTUAL, main='QQPlot of ACTUAL_PERIOD \n (Auto Mobile)', col='Navy', 
       xlab='Light Tailed')
qqline(final_join_data_EV$PERIOD_ACTUAL, col='red')

In [ ]:
# Mobile & IT
qqnorm(final_join_data_MITB$PERIOD_DAYS, main='QQPlot of PLANNING_PERIOD \n (Mobile & IT)', col='Navy', 
       xlab='Light Tailed')
qqline(final_join_data_MITB$PERIOD_DAYS, col='red')

qqnorm(final_join_data_MITB$PERIOD_ACTUAL, main='QQPlot of ACTUAL_PERIOD \n (Mobile & IT)', col='Navy', 
       xlab='Light Tailed')
qqline(final_join_data_MITB$PERIOD_ACTUAL, col='red')

#### *re-test: Shapiro-Wilk Test*

In [ ]:
# cannot reject H_0
shapiro.test(final_join_data_EV$PERIOD_ACTUAL) # p-value = 1.021e-05
shapiro.test(final_join_data_EV$PERIOD_DAYS) # p-value = 1.585e-06

shapiro.test(final_join_data_MITB$PERIOD_ACTUAL)# p-value < 2.2e-16
shapiro.test(final_join_data_MITB$PERIOD_DAYS) # p-value < 2.2e-16

### **6. time-series plot for Period Differences between planning period and actual period**

In [ ]:
final_join_data_all <- rbind(final_join_data_EV, final_join_data_MITB)
# View(final_join_data_all) # not supported in Jupyter R Kernel

# main (Automobile + Mobile & IT)
# change label name (buc_label) -> next trial!
time_series_plot_main <- ggplot(final_join_data_all, aes(RESULT_START_DATE, Diff_Period, colour=BUSINESS_UNIT_CODE, group=BUSINESS_UNIT_CODE))+ 
  geom_line(size=1.07) + geom_point(size=1)

datebreaks <- seq(min(final_join_data_all$RESULT_START_DATE), max(final_join_data_all$RESULT_START_DATE), by="12 month")

max(final_join_data_all$Diff_Period) # 812 > 562
min(final_join_data_all$Diff_Period) # -786 (-)

time_series_plot_main + 
  ggtitle("Time series, Period Difference btw Planning and Actual") +
  theme(plot.title=element_text(size=14, hjust = 0)) +
  xlab("date") + ylab("Period Diff btw Planning & Actual (days)") +
  labs(col = "사업부") + 
  scale_x_date(breaks=datebreaks) +
  scale_y_continuous(breaks = seq(-800, 600, 300)) +
  theme(axis.text.x = element_text(angle=30, hjust=1)) +
  scale_color_manual(labels=c("Automobile", "Mobile & IT"), values = c('Steel Blue', 'Navy'))

In [ ]:
# Automobile
ggplot(final_join_data_EV, aes(RESULT_START_DATE, Diff_Period))+ 
  geom_line(size=1.1, color = 'Navy') + 
  ggtitle("Time series, Differences between Planning and Actual Date") + 
  theme(plot.title=element_text(size=14))

In [ ]:
# Mobile & IT
ggplot(final_join_data_MITB, aes(RESULT_START_DATE, Diff_Period))+ 
  geom_line(size=1.1, color = 'Navy') + 
  ggtitle("Time series, Differences between Planning and Actual Date") + 
  theme(plot.title=element_text(size=14))

# View(final_join_data_EV) # not supported in Jupyter R Kernel

### **7. Count Plot MODEL_GRADE~ACTIVITY_TYPE_NAME**

In [ ]:
# Automobile
my.table_EV <- xtabs(~ ACTIVITY_TYPE_NAME+MODEL_GRADE, data = final_join_data_EV)
my.table_EV

# Mobile & IT
my.table_MITB <- xtabs(~ ACTIVITY_TYPE_NAME+MODEL_GRADE, data = final_join_data_MITB)
my.table_MITB

Countplot <- barplot(my.table,
                     xlab = 'MODEL_GRADE', 
                     ylab = 'Frequency', 
                     legend.text = TRUE)

#### *just count*

In [ ]:
buc_label <- c(EV = "Automobile", MITB = "Mobile & IT")

Countplot <- ggplot(final_join_data_all, aes(MODEL_GRADE, fill = ACTIVITY_TYPE_NAME)) + 
  geom_bar(stat = 'count', position = 'dodge' ,colour = 'Navy') + # position = 'dodge' 
  geom_text(aes(label = ..count..), stat = 'count', position = position_dodge(width = 1), vjust = -0.5, hjust = 0.5)

Countplot + 
  facet_grid(~BUSINESS_UNIT_CODE, labeller = labeller(BUSINESS_UNIT_CODE = buc_label)) +
  scale_fill_brewer(name = 'Gate') +
  xlab("개발 등급") + 
  ylab("count") +
  ggtitle("Count Plot") +
  scale_x_discrete(limits = c("S", "A", "B"))

# count + position = fill
Countplot <- ggplot(final_join_data_all, aes(ACTIVITY_TYPE_NAME, fill = MODEL_GRADE)) + geom_bar(stat = 'count', position = 'fill')
Countplot + 
  facet_grid(~BUSINESS_UNIT_CODE) +
  scale_fill_brewer(palette = 'Paired')

### **8. Scatter Plot**

#### **Automobile**
#### *for the ACTUAL_PERIOD*

In [ ]:
Scatterplot_actual_EV <- ggplot(data = final_join_data_EV,
                                aes(x = MODEL_GRADE, y = PERIOD_ACTUAL, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_EV
Scatterplot_actual_EV + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Actual Period) - Automobile") +
  xlab("개발 등급") + 
  ylab("실적 기간 (days)")

#### *for the PLANNED_PERIOD*

In [ ]:
Scatterplot_actual_EV <- ggplot(data = final_join_data_EV,
                                aes(x = MODEL_GRADE, y = PERIOD_DAYS, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_EV
Scatterplot_actual_EV + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Planning Period)") +
  xlab("개발 등급") + 
  ylab("계획 기간 (days)")

#### *for the DIFF_PERIOD*

In [ ]:
Scatterplot_actual_EV <- ggplot(data = final_join_data_EV,
                                aes(x = MODEL_GRADE, y = Diff_Period, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_EV
Scatterplot_actual_EV + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Period Difference btw Planning & Actual)") +
  xlab("개발 등급") + 
  ylab("Period Diff btw Planning & Actual (days)")

#### **Mobile & IT**
#### *for the ACTUAL_PERIOD*

In [ ]:
Scatterplot_actual_MITB <- ggplot(data = final_join_data_MITB,
                                  aes(x = MODEL_GRADE, y = PERIOD_ACTUAL, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_MITB
Scatterplot_actual_MITB + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Actual Period) - Mobile & IT") +
  xlab("개발 등급") + 
  ylab("실적 기간 (days)")

#### *for the PLANNING_PERIOD*

In [ ]:
Scatterplot_actual_MITB <- ggplot(data = final_join_data_MITB,
                                  aes(x = MODEL_GRADE, y = PERIOD_DAYS, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_MITB
Scatterplot_actual_MITB + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Planning Period)") +
  xlab("개발 등급") + 
  ylab("계획 기간 (days)")

#### *for the DIFF_PERIOD*

In [ ]:
Scatterplot_actual_MITB <- ggplot(data = final_join_data_MITB,
                                  aes(x = MODEL_GRADE, y = Diff_Period, color = MODEL_GRADE)) +
  geom_point(position = 'jitter') + facet_grid(~ACTIVITY_TYPE_NAME)

Scatterplot_actual_MITB
Scatterplot_actual_MITB + scale_color_manual(name = '개발 등급', values = c('Light Steel Blue', 'Royal Blue', 'Navy')) + 
  geom_boxplot(alpha = 0.1, color = 'black', fill = 'Navy') +
  scale_x_discrete(limits = c("S", "A", "B")) +
  ggtitle("Scatter Plot (Period Difference btw Planning & Actual)") +
  xlab("개발 등급") + 
  ylab("Period Diff btw Planning & Actual (days)")

## **Run the linear regression**

#### **Auto mobile Period Prediction Model**

#### *Basegroup B_CV is not included (g-1)*

In [ ]:
PERIOD_ACTUAL_EV <- final_join_data_EV$PERIOD_ACTUAL

S_CV_EV <- final_join_data_EV$S_CV
S_DV_EV <- final_join_data_EV$S_DV
S_PD_EV <- final_join_data_EV$S_PD
S_PV_EV <- final_join_data_EV$S_PV

A_CV_EV <- final_join_data_EV$A_CV
A_DV_EV <- final_join_data_EV$A_DV
A_PD_EV <- final_join_data_EV$A_PD
A_PV_EV <- final_join_data_EV$A_PV

B_CV_EV <- final_join_data_EV$B_CV # base group
B_DV_EV <- final_join_data_EV$B_DV
B_PD_EV <- final_join_data_EV$B_PD
B_PV_EV <- final_join_data_EV$B_PV

#### *first trial - only linear relationship*

In [ ]:
reg1.PERIOD_EV <- lm(PERIOD_ACTUAL_EV~S_CV_EV+S_DV_EV+S_PD_EV+S_PV_EV+
                       A_CV_EV+A_DV_EV+A_PD_EV+A_PV_EV+
                       B_DV_EV+B_PD_EV+B_PV_EV)
summary(reg1.PERIOD_EV)

#### *Second trial - Linear relationship & Delete biased variables (A_PV_EV: 0.99541, A_CV_EV: 0.5302)*

In [ ]:
reg1_1.PERIOD_EV <- lm(PERIOD_ACTUAL_EV~S_DV_EV+S_PD_EV+B_DV_EV)
summary(reg1_1.PERIOD_EV)

#### *Third trial - including non-linear relationship*

In [ ]:
# square, log
PERIOD_ACTUAL_EV_square <- (final_join_data_EV$PERIOD_ACTUAL)^2
PERIOD_ACTUAL_EV_ln <- log(final_join_data_EV$PERIOD_ACTUAL)
typeof(PERIOD_ACTUAL_EV_ln)

# Squared Form - B_CV_EV: Base group
reg2.PERIOD_EV <- lm(PERIOD_ACTUAL_EV_square~S_CV_EV+S_DV_EV+S_PD_EV+S_PV_EV+
                       A_CV_EV+A_DV_EV+A_PD_EV+A_PV_EV+
                       B_DV_EV+B_PD_EV+B_PV_EV)
summary(reg2.PERIOD_EV)

#### *3-1) Squared Form & Delete Biased Variables*

In [ ]:
reg2_1.PERIOD_EV <- lm(PERIOD_ACTUAL_EV_square~S_DV_EV+S_PD_EV+B_DV_EV)
summary(reg2_1.PERIOD_EV)

#### *3-2) Log Form - B_CV_EV: Base group*

In [ ]:
reg3.PERIOD_EV <- lm(PERIOD_ACTUAL_EV_ln~S_CV_EV+S_DV_EV+S_PD_EV+S_PV_EV+
                       A_CV_EV+A_DV_EV+A_PD_EV+A_PV_EV+A_PD_EV+
                       B_DV_EV+B_PD_EV+B_PV_EV)
summary(reg3.PERIOD_EV)
(88583)^(1/2)

#### *Second Trial - Log Form & Delete Biased Variables*

In [ ]:
reg3_1.PERIOD_EV <- lm(PERIOD_ACTUAL_EV_ln~S_CV_EV+S_DV_EV+S_PD_EV+
                         A_DV_EV+
                         B_DV_EV+B_PD_EV+B_PV_EV)
summary(reg3_1.PERIOD_EV)

### **Mobile & IT Period Prediction Model**
#### *Basegroup B_CV_MITB is not included (g-1)*
#### *first trial - only linear relationship*

In [ ]:
PERIOD_ACTUAL_MITB <- final_join_data_MITB$PERIOD_ACTUAL

S_CV_MITB <- final_join_data_MITB$S_CV
S_DV_MITB <- final_join_data_MITB$S_DV
S_PD_MITB <- final_join_data_MITB$S_PD
S_PV_MITB <- final_join_data_MITB$S_PV

A_CV_MITB <- final_join_data_MITB$A_CV
A_DV_MITB <- final_join_data_MITB$A_DV
A_PD_MITB <- final_join_data_MITB$A_PD
A_PV_MITB <- final_join_data_MITB$A_PV

B_CV_MITB <- final_join_data_MITB$B_CV # base group
B_DV_MITB <- final_join_data_MITB$B_DV
B_PD_MITB <- final_join_data_MITB$B_PD
B_PV_MITB <- final_join_data_MITB$B_PV
exp(5.0247)

reg1.PERIOD_MITB <- lm(PERIOD_ACTUAL_MITB~S_CV_MITB+S_DV_MITB+S_PD_MITB+S_PV_MITB+
                         A_CV_MITB+A_DV_MITB+A_PD_MITB+A_PV_MITB+
                         B_DV_MITB+B_PD_MITB+B_PV_MITB)
summary(reg1.PERIOD_MITB)

#### *Check the MLR Assumption*

In [ ]:
y_hat <- reg1.PERIOD_MITB$fitted.values
y <- PERIOD_ACTUAL_MITB
u_hat <- y - y_hat

hist(u_hat, breaks=25, probability=TRUE, main='PDF of Normal Distribution',
     col='Navy', border='white', xlab='Normal')
lines(density(u_hat), col='red')

qqnorm(u_hat, col='Navy')
qqline(u_hat, col='red')

#### *re-test: Shapiro-Wilk Test*

In [ ]:
# cannot reject H_0
shapiro.test(u_hat) # p-value = 1.021e-05
skewness(u_hat)

#### *Second trial - linear relationship & Delete biased variables (S_PD_MITB: 0.629415)*

In [ ]:
reg2.PERIOD_MITB <- lm(PERIOD_ACTUAL_MITB~S_CV_MITB+S_DV_MITB+S_PV_MITB+
                         A_CV_MITB+A_DV_MITB+A_PD_MITB+A_PV_MITB+
                         B_DV_MITB+B_PD_MITB+B_PV_MITB)
summary(reg2.PERIOD_MITB)

#### *Check the MLR Assumption*

In [ ]:
y_hat_2 <- reg2.PERIOD_MITB$fitted.values
y <- PERIOD_ACTUAL_MITB
u_hat_2 <- y - y_hat_2

hist(u_hat_2, breaks=25, probability=TRUE, main='PDF of Normal Distribution',
     col='Navy', border='white', xlab='Normal')
lines(density(u_hat_2), col='red')

qqnorm(u_hat_2, col='Navy')
qqline(u_hat_2, col='red')

#### *re-test: Shapiro-Wilk Test*

In [ ]:
# cannot reject H_0
shapiro.test(u_hat_2) # p-value = 1.021e-05
skewness(u_hat_2)

#### *Non-linear Relationship > Next Trial!*

In [ ]:
PERIOD_ACTUAL_MITB_ln <- log(PERIOD_ACTUAL_MITB) # -Inf
PERIOD_ACTUAL_MITB_sq <- (PERIOD_ACTUAL_MITB)^2

#### *Third trial - non-linear relationship(squared)*

In [ ]:
reg3.PERIOD_MITB <- lm(PERIOD_ACTUAL_MITB_sq~S_CV_MITB+S_DV_MITB+S_PV_MITB+S_PD_MITB+
                         A_CV_MITB+A_DV_MITB+A_PD_MITB+A_PV_MITB+
                         B_DV_MITB+B_PD_MITB+B_PV_MITB)
summary(reg3.PERIOD_MITB)

#### *Forth trial - non-linear relationship(log)*

In [ ]:
reg4.PERIOD_MITB <- lm(PERIOD_ACTUAL_MITB_ln~S_CV_MITB+S_DV_MITB+S_PV_MITB+S_PD_MITB+
                         A_CV_MITB+A_DV_MITB+A_PD_MITB+A_PV_MITB+
                         B_DV_MITB+B_PD_MITB+B_PV_MITB)
summary(reg4.PERIOD_MITB)

reg4_1.PERIOD_MITB <- lm(PERIOD_ACTUAL_MITB_ln~S_CV_MITB+S_DV_MITB+S_PV_MITB+
                         A_CV_MITB+A_DV_MITB+A_PD_MITB+A_PV_MITB+
                         B_DV_MITB+B_PD_MITB+B_PV_MITB)
summary(reg4_1.PERIOD_MITB)

#### *Check the MLR Assumption*

In [ ]:
y_hat_4_1 <- reg4_1.PERIOD_MITB$fitted.values
y <- PERIOD_ACTUAL_MITB
u_hat_4_1 <- y - y_hat_4_1

hist(u_hat_4_1, breaks=25, probability=TRUE, main='PDF of Normal Distribution',
     col='Navy', border='white', xlab='Normal')
lines(density(u_hat_4_1), col='red')

qqnorm(u_hat_4_1, col='Navy')
qqline(u_hat_4_1, col='red')

#### *re-test: Shapiro-Wilk Test*

In [ ]:
# cannot reject H_0
shapiro.test(u_hat_4_1) # p-value = 1.021e-05
skewness(u_hat_4_1) # Gate 단계별로 분포를 가짐

## **Write the final excel file on your local PC**

In [ ]:
write_xlsx(final_join_data_MITB, path="C://Users//LG//Desktop//IBT//배치부서특화//과제_8월//WBS일정계획//raw data//2차추출_all//final_data_MITB_add.xlsx")
write_xlsx(final_join_data_EV, path="C://Users//LG//Desktop//IBT//배치부서특화//과제_8월//WBS일정계획//raw data//2차추출_all//final_data_EV_add.xlsx")